## QR Algorithm

**Tasks**
1. Using Householder method, convert matrix
$$
\boldsymbol{A} = \begin{bmatrix}
1 & 2 & 3 \\ 2 & 4 & -3 \\ 3 & -3 & 4
\end{bmatrix}
$$
to a Hessenberg form.

2. Use QR algorithm to the resultant matrix and obtain eigenvalues.

### Task 1.

Define $\boldsymbol{A}_1:=\boldsymbol{A}$.

For each $k=1,\dots,n-2$ we make the following steps. We build the following vectors:
$$
\boldsymbol{a} = \begin{bmatrix}
0 \\ \vdots \\ 0 \\ (\boldsymbol{A}_k^{[k]})_{k+1} \\ \vdots \\ (\boldsymbol{A}_k^{[k]})_{n}
\end{bmatrix}, \; \boldsymbol{b} = \|\boldsymbol{a}\|_2\boldsymbol{e}_{k+1}, \; \boldsymbol{v} = \begin{cases}
\frac{\boldsymbol{a}-\boldsymbol{b}}{\|\boldsymbol{a}-\boldsymbol{b}\|_2}, \boldsymbol{a} \neq \boldsymbol{b} \\
0, \boldsymbol{a} = \boldsymbol{b}
\end{cases}
$$

And matrices:
$$
\boldsymbol{U} = \boldsymbol{E} - 2\boldsymbol{v}\boldsymbol{v}^{\top}, \; \boldsymbol{A}_{k+1}=\boldsymbol{U}_k\boldsymbol{A}_k\boldsymbol{U}_k
$$

In [38]:
# Importing scipy and numpy and setting print options
import numpy as np
import scipy.linalg as scl

np.set_printoptions(suppress=True, precision=3)

In [39]:
def to_hessenberg(A):
    """
    to_hessenberg converts the specified matrix to a hessenberg form.
    
    Input:
    A - square matrix
    
    Output:
    matrix A in the Hessenberg form
    """
    n = len(A) # Size of matrix A
    E = np.identity(n) # Identity matrix
    A = A.copy() # Copying the matrix A
    for k in range(n-1):
        v = A[:,k].copy() # Taking a kth column of matrix A
        v[:k+1] = 0 # Making 0 on all position below (k+1)th (or above if we consider v as a vector)
        v[k+1] = v[k+1] - np.linalg.norm(v, 2)  # From the element a_{k,k+1} we subtract norm of v
        v = v / np.linalg.norm(v, 2) # Normalizing vector v
        U = E - 2 * v.reshape(n,1).dot(v.reshape(1,n)) # Finding U_{k} = E - 2v*v^T
        A = U.dot(A).dot(U) # Finding A_{k+1} = U_{k}A{k}U_{k}
    return A

In [49]:
A = np.array([
    [1, 2, 3],
    [2, 4, -3],
    [3, -3, 4]
], dtype=np.float64) # Defining our matrix

H = to_hessenberg(A) # Converting A to a hessenberg form
print('Hessenberg form of A is\n{}'.format(H)) # Printing a result

Hessenberg form of A is
[[1.    3.606 0.   ]
 [3.606 1.231 1.154]
 [0.    1.154 6.769]]


Let us check the result using built in function in SciPy $\texttt{scl.hessenberg}$

In [50]:
H_scl = scl.hessenberg(A, calc_q=False)
print('Hessenberg form of A using scipy is\n{}'.format(H_scl))

Hessenberg form of A using scipy is
[[ 1.    -3.606  0.   ]
 [-3.606  1.231 -1.154]
 [ 0.    -1.154  6.769]]


### Task 2.

Define $\boldsymbol{A}_1=\boldsymbol{A}$. On each step $k \geq 1$ we do:
1. Find QR decomposition of $\boldsymbol{A}_k$: $\boldsymbol{A}_k = \boldsymbol{Q}_k\boldsymbol{R}_k$.
2. Set $\boldsymbol{A}_{k+1}=\boldsymbol{R}_k\boldsymbol{Q}_k$

In [51]:
def qr_algorithm(A, steps_number=30):
    """
    qr_algorithm returns a form of matrix A from which it is easy to retrieve eigenvalues
    
    Input:
    A - specified matrix
    steps_number - number of steps to complete
    
    Output:
    modified matrix A
    """
    M = A.copy() # Copying the matrix A
    for i in range(steps_number):
        Q, R = np.linalg.qr(M) # Finding QR decomposition of A
        M = R.dot(Q) # Finding A_{k+1}=R_{k}Q_{k}
    return M

In [53]:
for i in range(6):
    steps_number = 5*i # We make 20i steps
    qr_A = qr_algorithm(H, steps_number=steps_number)
    print('After applying QR algorithm with {} steps, we obtain a matrix\n{}'.format(steps_number, qr_A))

After applying QR algorithm with 0 steps, we obtain a matrix
[[1.    3.606 0.   ]
 [3.606 1.231 1.154]
 [0.    1.154 6.769]]
After applying QR algorithm with 5 steps, we obtain a matrix
[[ 6.78  -0.916 -0.   ]
 [-0.916  4.365  1.687]
 [-0.     1.687 -2.146]]
After applying QR algorithm with 10 steps, we obtain a matrix
[[ 7.119 -0.096  0.   ]
 [-0.096  4.441  0.107]
 [-0.     0.107 -2.56 ]]
After applying QR algorithm with 15 steps, we obtain a matrix
[[ 7.122 -0.009 -0.   ]
 [-0.009  4.439  0.007]
 [-0.     0.007 -2.562]]
After applying QR algorithm with 20 steps, we obtain a matrix
[[ 7.123 -0.001  0.   ]
 [-0.001  4.439  0.   ]
 [-0.     0.    -2.562]]
After applying QR algorithm with 25 steps, we obtain a matrix
[[ 7.123 -0.    -0.   ]
 [-0.     4.439  0.   ]
 [-0.     0.    -2.562]]


Thus we see that after applying a $QR$ algorithm we get eigenvalues $\lambda_1\approx 7.123,\lambda_2\approx 4.439,\lambda_3\approx -2.562$.